<h1>Saxon Pacing Log PDF Reader</h1>

<h3>python</h3>
<p>3.8</p>
<h3>requirements</h3>
<ul>
    <li>numpy==1.18.5
    <li>pandas==1.2.4
    <li>PyPDF2==1.26.0
    <li>termcolor==1.1.0
</ul>

In [5]:
# Imports

import pandas as pd
import numpy as np
import PyPDF2
import re
from termcolor import colored

In [14]:
# Document Reader

def find_vals(path, ):
    pdfFileObj = open(path, 'rb')
    pdfReader=PyPDF2.PdfFileReader(pdfFileObj)

    allPages = ""
    for x in range(pdfReader.numPages):
        allPages=allPages+str(pdfReader.getPage(x).extractText())
    
    z = re.findall('LESSON(.+?)OBJECTIVES(.+?)Math Center',allPages.replace('\n', ' ').replace('•', '|'))  #specify functional regex
    
    lct = len(re.findall('LESSON', str(allPages))) #verification regex
    
    print(f"{path}\nLessons: {len(re.findall('LESSON', str(allPages)))}\nLessons Matched: {len(z)}\n\t{colored('Pass', 'green') if  lct==len(z) else colored('You may have missed values. Please verify regex!', 'red')}") #validate 
    
    
    ls = []
    for tup in z:
        ls.append([tup[0],'; '.join(tup[1].replace('™', "'").replace('ﬁ', '"').replace('ﬂ', '"').replace('Œ', '-').split('|'))])

    df = pd.DataFrame({'lesson_code':dict(ls).keys(), 'description':dict(ls).values()})
    df.description = df.description.apply(lambda x: x[2:].strip() if x.startswith(' ;') | x.startswith(';') else x.strip()) # remove leading ;
    df.description = df.description.apply(lambda x: x[:3].replace(' ', '')+x[3:]) # fix 'I dentify' disjointed leading word issue
    
    return df

In [15]:
df_k=find_vals('R:\\Data\\Data Dept\\Database\\Data Imports\\2021-22\\curriculum\Kinder Saxon Lesson Guide.pdf')
df_1=find_vals('R:\\Data\\Data Dept\\Database\\Data Imports\\2021-22\\curriculum\First Saxon Lesson Guide.pdf')
df_2=find_vals('R:\\Data\\Data Dept\\Database\\Data Imports\\2021-22\\curriculum\Second Saxon Lesson Guide.pdf')

R:\Data\Data Dept\Database\Data Imports\2021-22\curriculum\Kinder Saxon Lesson Guide.pdf
Lessons: 146
Lessons Matched: 146
	Pass
R:\Data\Data Dept\Database\Data Imports\2021-22\curriculum\First Saxon Lesson Guide.pdf
Lessons: 164
Lessons Matched: 164
	Pass
R:\Data\Data Dept\Database\Data Imports\2021-22\curriculum\Second Saxon Lesson Guide.pdf
Lessons: 163
Lessons Matched: 163
	Pass


In [11]:
# Add program code

df_k['program_code'] = 'Saxon K'
df_1['program_code'] = 'Saxon 1'
df_2['program_code'] = 'Saxon 2'

In [12]:
df_all = pd.concat([df_k, df_1, df_2], ignore_index=True) # stack data frames
df_all['sort_order'] = df_all.index+1  # Add sort order

In [7]:
# Export as tab separated

df_all.loc[:,['program_code', 'lesson_code', 'description', 'sort_order']].to_csv('R:\\Data\\Data Dept\\Database\\Data Imports\\2022-23\\curriculum\myFileName.csv', index=False, sep='\t')